In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
sample_data_path = '/content/drive/MyDrive/Colab Notebooks'
files=os.listdir(sample_data_path)
print(files)

In [ ]:
ff='/content/drive/MyDrive/Colab Notebooks/featured_shihara.xlsx'

In [ ]:
import pandas as pd
df=pd.read_excel(ff,engine='openpyxl')

In [ ]:
print(df.head())
print(df.dtypes)

In [ ]:
df["Date"]=pd.to_datetime(df["Date"])

In [ ]:
nf_df=df

In [ ]:
# Required libraries
!pip install neuralforecast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATSx
from neuralforecast.utils import AirPassengersDF
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
# Data preparation: Split into train (90%) and test (10%)
train_size = int(len(nf_df) * 0.9)
train_df = nf_df.iloc[:train_size].copy()
test_df = nf_df.iloc[train_size:].copy()

print(f"Training set size: {len(train_df)}")
print(f"Testing set size: {len(test_df)}")

# Handle missing values in both datasets
train_df = train_df.fillna(method='bfill').fillna(method='ffill')
test_df = test_df.fillna(method='bfill').fillna(method='ffill')

# Set up the data in NeuralForecast format
train_data = train_df.copy()
train_data['unique_id'] = 'balance'  # Add required ID column
train_data = train_data.rename(columns={'Date': 'ds', 'Normalized_Balance': 'y'})

test_data = test_df.copy()
test_data['unique_id'] = 'balance'
test_data = test_data.rename(columns={'Date': 'ds', 'Normalized_Balance': 'y'})

In [ ]:
# Define exogenous variables (all features except date, target and ID)
exogenous_vars = [col for col in train_data.columns
                  if col not in ['ds', 'y', 'unique_id']]

print(f"Using {len(exogenous_vars)} exogenous variables: {exogenous_vars}")

# Define forecast horizon
horizon = 30

# Create and train the NBEATSx model
# Note: Fixed version with proper parameter setup
model = NBEATSx(
    h=horizon,                   # Forecast horizon
    input_size=2*horizon,        # Input window length
    futr_exog_list=exogenous_vars,  # Use future exogenous variables if available
    hist_exog_list=exogenous_vars,  # Use historical exogenous variables
    random_seed=42,              # For reproducibility
    scaler_type='standard'       # Standardize the data
)

# Create the forecaster
forecaster = NeuralForecast(
    models=[model],
    freq='D'  # Daily frequency
)

# Fit the model (train)
forecaster.fit(df=train_data)

# Generate forecasts
forecast_df = forecaster.predict(
    futr_df=test_data.iloc[:horizon]  # First 30 days of test data for comparison
)

In [ ]:
# Extract actual values and forecasts
actual = test_data['y'].iloc[:horizon].values
forecast = forecast_df.loc[forecast_df['unique_id'] == 'balance', 'NBEATSx'].values

# Plot results
plt.figure(figsize=(12, 6))
plt.plot(range(horizon), actual, label='Actual', marker='o', alpha=0.7)
plt.plot(range(horizon), forecast, label='NBEATSx Forecast', marker='x', alpha=0.7)
plt.title('30-Day Balance Forecast vs Actual')
plt.xlabel('Days')
plt.ylabel('Normalized Balance')
plt.legend()
plt.grid(True)
plt.tight_layout()

# Calculate error metrics
mae = mean_absolute_error(actual, forecast)
rmse = np.sqrt(mean_squared_error(actual, forecast))
# Add small epsilon to avoid division by zero in MAPE calculation
mape = np.mean(np.abs((actual - forecast) / (actual + 1e-8))) * 100

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

plt.show()

In [ ]:
# Generate a more comprehensive visualization showing training history and forecast
# Take the last 60 days of training data for context
history_size = 60
historical_data = train_data.iloc[-history_size:]['y'].values
forecast_dates = range(history_size, history_size + horizon)
history_dates = range(history_size)

plt.figure(figsize=(14, 7))
# Plot historical data
plt.plot(history_dates, historical_data, label='Historical Balance', color='blue', alpha=0.6)
# Plot actual test data
plt.plot(forecast_dates, actual, label='Actual Balance', color='green', marker='o', alpha=0.7)
# Plot forecasted data
plt.plot(forecast_dates, forecast, label='NBEATSx Forecast', color='red', marker='x', alpha=0.7)
# Add vertical line to mark train/test split
plt.axvline(x=history_size, color='black', linestyle='--', label='Train/Test Split')

plt.title('Balance History and 30-Day Forecast')
plt.xlabel('Days')
plt.ylabel('Normalized Balance')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
!jupyter nbconvert --ClearOutputPreprocessor.enabled=True --inplace cleaned_no_widgets.ipynb
